爬CSI300成分股

In [3]:
import baostock as bs
import pandas as pd


lg = bs.login()

print('login respond error_code:' + lg.error_code)
print('login respond error_msg:' + lg.error_msg)

file_name = "G:/hs300_stocks.csv" 

date = '2018-07-31'

hs300_stocks = []
rs = bs.query_hs300_stocks(date)

while (rs.error_code == '0') & rs.next():  
    hs300_stocks.append(rs.get_row_data())

result = pd.DataFrame(hs300_stocks, columns=rs.fields)  
bs.logout()


login success!
login respond error_code:0
login respond error_msg:success
logout success!


爬成分股 Adjust closing price

In [ ]:
import yfinance as yf

def convert_code(code):
    if code.startswith("sh."):
        return code.split(".")[1] + ".SS"
    if code.startswith("sz."):
        return code.split(".")[1] + ".SZ"
    else:
        return code

yahoo_codes = [convert_code(code) for code in result['code']]

start_date = '2006-01-04'
end_date = '2019-12-31'

adjusted_prices = {}

def fetch_adjusted_closing_prices(symbol, start_date, end_date):
    try:
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        adjusted_prices[symbol] = stock_data['Adj Close']
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

for symbol in yahoo_codes:
    fetch_adjusted_closing_prices(symbol, start_date, end_date) 

In [25]:
adjusted_prices_df = pd.DataFrame(adjusted_prices)
adjusted_prices_df = adjusted_prices_df.dropna(axis=1)
adjusted_prices_df.head()

adjusted_prices_df.index = pd.to_datetime(adjusted_prices_df.index)
years = adjusted_prices_df.index.year.unique()
sub_dfs = {year: adjusted_prices_df[adjusted_prices_df.index.year == year] for year in years}

In [153]:
sub_dfs[2006].head()

,600000.SS,600008.SS,600009.SS,600010.SS,600011.SS,600015.SS,600016.SS,600018.SS,600019.SS,600028.SS,...,000961.SZ,000963.SZ,000983.SZ,001979.SZ,002007.SZ,002008.SZ,002024.SZ,002027.SZ,002044.SZ,002050.SZ
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,1.044573,1.304761,6.167114,0.620546,2.669785,1.169139,0.652753,6.868121,1.911539,1.630682,...,0.439580,0.688773,1.104152,3.815096,0.524399,1.669167,1.006467,1.133809,0.602338,0.634960
2006-01-05,1.061898,1.355209,6.784245,0.628894,2.732394,1.204640,0.652753,7.090071,1.934681,1.630682,...,0.441214,0.698496,1.088601,3.858877,0.542293,1.703519,0.990637,1.144589,0.607928,0.640115
2006-01-06,1.092471,1.357503,6.679292,0.640025,2.808418,1.204640,0.654289,7.083905,1.920796,1.623816,...,0.441214,0.711461,1.135255,3.858877,0.541796,1.734747,0.985530,1.155371,0.605133,0.646990
2006-01-09,1.079222,1.357503,6.679292,0.645590,2.763697,1.209373,0.648145,6.960598,1.911539,1.610084,...,0.442849,0.726047,1.141087,3.858877,0.554222,1.775345,1.031488,1.180527,0.619109,0.669329
2006-01-10,1.082280,1.387312,6.679292,0.665069,2.710033,1.223573,0.646609,6.757144,1.888397,1.623816,...,0.442849,0.726047,1.135255,3.858877,0.553726,1.762853,1.030977,1.166152,0.613518,0.663314


ADF test，EG test

In [123]:
from arch.unitroot import engle_granger
from arch.unitroot import ADF
import pandas as pd
import numpy as np

lambda_thresholds = (0.1, 0.05, 0.01, 0.005)

all_cointegrated_pairs = {thresh: {} for thresh in lambda_thresholds}

for threshold in lambda_thresholds:
    for year, df in sub_dfs.items():
        columns = df.columns
        cointegrated_pairs = []

        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                column_i = columns[i]
                column_j = columns[j]

                series_i = df[column_i]
                series_j = df[column_j]

                try:
                    eg_test = engle_granger(series_i, series_j, trend='c')

                    if eg_test.pvalue < threshold:
                        cointegrated_pairs.append((column_i, column_j, eg_test.pvalue))

                except Exception as e:
                    continue  
                
        if cointegrated_pairs:
            all_cointegrated_pairs[threshold][year] = cointegrated_pairs


In [154]:
 all_cointegrated_pairs[0.01][2006]

[('600000.SS', '600690.SS', 0.0036101928689708266),
 ('600008.SS', '000157.SZ', 0.0078000647980169735),
 ('600009.SS', '600588.SS', 0.009984254888181903),
 ('600011.SS', '000001.SZ', 0.0014627744962193527),
 ('600011.SS', '000069.SZ', 0.007602540282193803),
 ('600016.SS', '600690.SS', 0.008070724424655505),
 ('600016.SS', '000069.SZ', 0.0007681267730677687),
 ('600019.SS', '600663.SS', 0.00983957444766567),
 ('600019.SS', '600900.SS', 0.0035173832087683197),
 ('600028.SS', '600309.SS', 0.006450254008683806),
 ('600030.SS', '600583.SS', 0.0008321992009539862),
 ('600030.SS', '000876.SZ', 0.007905940012315963),
 ('600030.SS', '002024.SZ', 0.0008703722948103667),
 ('600031.SS', '600438.SS', 0.001109987084354238),
 ('600031.SS', '600585.SS', 0.008836687112888441),
 ('600038.SS', '600061.SS', 0.003754598968093894),
 ('600038.SS', '600233.SS', 0.0006191215037739814),
 ('600038.SS', '600704.SS', 0.00334438916209286),
 ('600066.SS', '600176.SS', 0.003863879189320168),
 ('600066.SS', '600276.SS

In [145]:
Iso = {threshold: {} for threshold in lambda_thresholds}
Edges = {threshold: {} for threshold in lambda_thresholds}

for threshold, years_data in all_cointegrated_pairs.items():
    adjacency_matrix_template = pd.DataFrame(0, index=adjusted_prices_df.columns, columns=adjusted_prices_df.columns)
    
    for year, pairs in years_data.items():

        adjacency_matrix = adjacency_matrix_template.copy()
        
        for stock_i, stock_j, pvalue in pairs:
            adjacency_matrix.at[stock_i, stock_j] = 1
            adjacency_matrix.at[stock_j, stock_i] = 1
        
        iso = (adjacency_matrix.sum(axis=1) == 0).sum()
        edge = adjacency_matrix.sum().sum() // 2
        
        Iso[threshold][year] = iso
        Edges[threshold][year] = edge

計算每張netwrok的拓撲性質

In [147]:
stats_columns = ['Min ISO', 'Max ISO', 'Mean ISO', 'Min Edges', 'Max Edges', 'Mean Edges']
stats_df = pd.DataFrame(index=lambda_thresholds, columns=stats_columns)

for threshold in lambda_thresholds:
    iso_values = list(Iso[threshold].values())
    edges_values = list(Edges[threshold].values())

    stats_df.loc[threshold, 'Min ISO'] = min(iso_values)
    stats_df.loc[threshold, 'Max ISO'] = max(iso_values)
    stats_df.loc[threshold, 'Mean ISO'] = np.mean(iso_values)

    stats_df.loc[threshold, 'Min Edges'] = min(edges_values)
    stats_df.loc[threshold, 'Max Edges'] = max(edges_values)
    stats_df.loc[threshold, 'Mean Edges'] = np.mean(edges_values)

stats_df

,Min ISO,Max ISO,Mean ISO,Min Edges,Max Edges,Mean Edges
0.100,0,5,1.285714,971,4608,1822.357143
0.050,0,11,4.714286,500,3491,1120.714286
0.010,1,52,30.357143,104,1812,403.571429
0.005,2,85,50.928571,50,1335,265.214286
